In [0]:
CATALOG = ""
SCHEMA = ""

In [ ]:
spark.sql(f"CREATE CATALOG IF NOT EXISTS {CATALOG};")
spark.sql(f"USE CATALOG {CATALOG};")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {SCHEMA};")
spark.sql(f"USE SCHEMA {SCHEMA};")

In [0]:
import pandas as pd

customers = pd.read_csv("https://github.com/marianreuss/fsi-genie/raw/refs/heads/main/customers_sampled.csv")
customers = spark.createDataFrame(customers)
customers.write.mode("overwrite").table("customers")

In [0]:
import pandas as pd

esg = pd.read_csv("https://github.com/marianreuss/fsi-genie/raw/refs/heads/main/esg_sampled.csv")
esg = spark.createDataFrame(esg)
esg.write.mode("overwrite").table("esg")

In [ ]:
spark.sql(f"""%sql
CREATE OR REPLACE FUNCTION cluster_byindustry(industry STRING)
RETURNS TABLE
LANGUAGE SQL
COMMENT 'returns companies with clustered environment grade'
RETURN SELECT
  `esg`.`ticker`,
  `esg`.`name`,
  `esg`.`currency`,
  `esg`.`exchange`,
  `esg`.`industry`,
  `esg`.`environment_grade`,
  `esg`.`environment_level`,
  `esg`.`social_grade`,
  `esg`.`social_level`,
  `esg`.`governance_grade`,
  `esg`.`governance_level`,
  `esg`.`environment_score`,
  `esg`.`social_score`,
  `esg`.`governance_score`,
  `esg`.`total_score`,
  `esg`.`total_grade`,
  `esg`.`total_level`,
  `esg`.`cik`,
  `customers`.`point_of_contact`,
  `customers`.`telephone_number`,
  `customers`.`email`,
  CASE
    WHEN `esg`.`environment_grade` = 'AA' THEN 0
    WHEN `esg`.`environment_grade` = 'A' THEN 1
    WHEN `esg`.`environment_grade` = 'BBB' THEN 2
    WHEN `esg`.`environment_grade` = 'BB' THEN 3
    WHEN `esg`.`environment_grade` = 'B' THEN 4
    ELSE 5
  END AS `cluster`
FROM
  `{CATALOG}`.`{SCHEMA}`.`esg`
  LEFT JOIN `{CATALOG}`.`{SCHEMA}`.`customers` ON `esg`.`ticker` = `customers`.`ticker`
WHERE LOWER(`esg`.`industry`) = LOWER(cluster_byindustry.industry)""")

In [ ]:

import requests

json_str = requests.get("https://github.com/marianreuss/fsi-genie/raw/refs/heads/main/dashboard.json").text


json_str = json_str.replace("placeholder_esg_table", f"{CATALOG}.{SCHEMA}.esg")
json_str = json_str.replace("placeholder_customer_table", f"{CATALOG}.{SCHEMA}.customers")
json_bytes = bytes(json_str.encode("utf-8"))

'{\n    "datasets": [\n        {\n            "name": "temp_dataset_name",\n            "displayName": "customers",\n            "query": "SELECT * placeholder_customer_table"\n        },\n        {\n            "name": "f7e37818",\n            "displayName": "esg",\n            "query": "SELECT * FROM placeholder_esg_table"\n        },\n        {\n            "name": "5a7ba337",\n            "displayName": "environment_score_by_industry",\n            "query": "select avg(environment_score) AS avg_environment_score, industry from placeholder_esg_table group by industry"\n        }\n    ],\n    "pages": [\n        {\n            "name": "temp_page_name",\n            "displayName": "New Page",\n            "layout": [\n                {\n                    "widget": {\n                        "name": "temp_bar_chart_name",\n                        "queries": [\n                            {\n                                "name": "main_query",\n                                "query"

In [ ]:
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.workspace import ImportFormat

client = WorkspaceClient()


workspace_file_path = f"/Workspace{dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()}"
workspace_file_path = "/".join(workspace_file_path.split("/")[:-1])


client.workspace.upload(
    path=workspace_file_path + "/fsi-genie-dashboard.lvdash.json",
    content=json_bytes,
    format=ImportFormat.AUTO,
    overwrite=False,
)